# Análise Exploratória dos Dados dos Conglomerados Pruendenciais

O Banco Central do Brasil disponibiliza dados das instituições sob sua supervisão, porém esses dados não são apresentados de maneira clara e objetiva. Embora estejam no formato de valores separados por vírgulas (CSV), eles ainda requerem um tratamento prévio para ser possível extrair informações relevantes de forma eficaz.

Neste notebook, realizaremos uma análise exploratória dos dados referentes aos conglomerados prudenciais e instituições independentes. Um conglomerado prudencial, conforme a Resolução BCB nº 168, de 1º de dezembro de 2021, é definido como um grupo composto por uma instituição de financeira que detém controle, direto ou indireto, sobre outras entidades no Brasil ou no exterior. Essas entidades podem incluir instituições financeiras, outras instituições autorizadas a funcionar pelo Banco Central do Brasil (BCB), instituições de pagamento não autorizadas pelo BCB, entidades que realizam aquisições de operações de crédito ou de direitos creditórios, além de outras pessoas jurídicas que tenham por objetivo exclusivo a participação societária nessas entidades e fundos de investimento. O controle é caracterizado quando a instituição investidora tem poder sobre a entidade controlada, podendo influenciar seus retornos e decisões. A instituição líder do conglomerado é responsável pela consolidação das demonstrações financeiras do grupo, e o Banco Central pode, a seu critério, incluir ou excluir entidades do conglomerado para garantir a correta representação do patrimônio consolidado.

In [58]:
# Importando as bibliotecas necessárias.

# Pandas: manipulação e análise de dados.
import pandas as pd

In [59]:
# Configurando o Pandas para exibir mais dados.
# Exibir todas as colunas (padrão é 20).
pd.set_option('display.max_columns', None)

# Definir largura máxima da coluna (padrão é 50).
pd.set_option('display.max_colwidth', 60)
# Definir precisão de exibição para números flutuantes.
pd.set_option('display.float_format', '{:.2f}'.format)

# Reduzir a quantidade de dados exibidos em DataFrames muito grandes.
pd.set_option('display.large_repr', 'truncate')

In [60]:
# Definindo o caminho do arquivo a ser carregado.
DATA_PATH = '../if_data_processed'
INSTITUTION_TYPE = 'prudential_conglomerates'
REPORT_TYPE = 'summary'
DATA_BASE = '2024-06.csv'
FILE_PATH = f'{DATA_PATH}/{INSTITUTION_TYPE}/{REPORT_TYPE}/{DATA_BASE}'

In [61]:
# Definindo os nomes das colunas do DataFrame, conforme a descrição das colunas.
COLUMN_NAMES = [
    'Instituicao',  # Nome da instituição ou conglomerado no cadastro do Banco Central.
    'Codigo',  # Código da instituição ou conglomerado no cadastro do Banco Central.
    'TipoConsBancario',  # Tipo de Consolidado Bancário.
    'Segmento',  # Segmento conforme Resolução n.º 4.553/2017.
    'TipoConsolidacao',  # Tipo de Consolidação (I) Instituição Independente, (C) Conglomerado.
    'TipoControle',  # Tipo de Controle (1) Público, (2) Privado Nacional, (3) Controle Estrangeiro.
    'Cidade',  # # Cidade da sede da instituição.
    'UF',  # Unidade da Federação onde fica a sede da instituição.
    'DataBase',  # Data-base do relatório.
    'AtivoTotal',  # Ativo Circulante e Realizável a Longo Prazo + Ativo Permanente.
    'CarteiraCredito',  # Carteira de Crédito Classificada.
    'PassivoExigivel',  # Passivo Circulante e Exigível a Longo Prazo e Resultado de Exercícios Futuros.
    'Captacoes',  # Captações de depósitos + Obrigações por Operações Compromissadas 
                  # + Recursos de Aceites Cambiais, Letras Imobiliárias e Hipotecárias,
                  # Debêntures e Similares + Obrigações por Empréstimos e Repasses.
    'PatrimonioLiquido',  # Patrimônio Líquido + Contas de Resultado Credoras
                          # + Contas de Resultado Devedoras. 
    'LucroLiquido',  # Lucro Líquido, excluindo despesas de juros sobre capital.
    'PatrimonioReferencia',  # Montante de capital regulatório formado pela soma
                             # das parcelas de Capital Nível I e Capital Nível II.
    'IndiceBasileia',  # Relação entre o Patrimônio de Referência e Ativos ponderados pelo risco.
    'IndiceImobilizacao',  # Relação entre Ativo Permanente e Patrimônio de Referência.
    'NumAgencias',  # Número de agências incluídas as sedes
    'NumPostosAtendimento'  # Número de postos de atendimento da instituição ou conglomerado
]

In [62]:
# Definindo as descrições dos valores das colunas tipo de consolidado bancário.
descricoes_tipo_consolidado = {
    "b1": "Banco Comercial ou Conglomerado com Carteira Comercial",
    "b2": "Banco Múltiplo sem Carteira Comercial ou Banco de Câmbio/Investimento",
    "b3S": "Cooperativa de Crédito Singular",
    "b3C": "Central e Confederação de Cooperativas de Crédito",
    "b4": "Banco de Desenvolvimento",
    "n1": "Instituição Não Bancária no Mercado de Crédito",
    "n2": "Instituição Não Bancária no Mercado de Capitais",
    "n4": "Instituição de Pagamento"
}

# Definindo as descrições dos valores das colunas tipo de consolidação.
descricoes_tipo_consolidacao = {
    "I": "Instituição Independente",
    "C": "Conglomerado"
}

# Definindo as descrições dos valores das colunas tipo de controle.
descricoes_tipo_controle = {
    1: "Público",
    2: "Privado Nacional",
    3: "Controle Estrangeiro"
}

# Definindo as regiões e estados do Brasil.
regioes_estados = {
    'Norte': ['AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO'],  # Ordem alfabética
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],  # Ordem alfabética
    'Sul': ['PR', 'RS', 'SC'],  # Ordem alfabética
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],  # Ordem alfabética
    'Centro-oeste': ['DF', 'GO', 'MT', 'MS']  # Ordem alfabética
}
# Mapeando os estados para as regiões do Brasil.
# Formato: {estado: regiao}
estado_para_regiao = {estado: regiao for regiao, estados in regioes_estados.items() for estado in estados}

In [63]:
# Carregando a base de dados para o DataFrame.
# Importante: O arquivo CSV está separado por ponto e vírgula.
df = pd.read_csv(FILE_PATH,
                 sep=';',
                 names=COLUMN_NAMES)

In [64]:
# Primeiro contato com o DataFrame.
df

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
0,ITAU - PRUDENCIAL,1000080099,b1,S1,C,2,SAO PAULO,SP,06/2024,2.636.882.506,946.459.806,2.443.588.602,1.884.371.314,193.293.904,19.963.578,215.556.905,"16,56%","20,86%",2.15,1.10
1,BB - PRUDENCIAL,1000080329,b1,S1,C,1,BRASILIA,DF,06/2024,2.334.367.943,1.024.504.848,2.161.635.722,1.845.389.366,172.732.221,18.173.027,175.348.301,"14,19%","16,35%",4.01,512.00
2,CAIXA ECONÔMICA FEDERAL - PRUDENCIAL,1000080738,b1,S1,C,1,BRASILIA,DF,06/2024,1.907.044.008,1.175.742.235,1.807.735.638,1.617.770.675,99.308.370,5.300.681,129.887.763,"16,40%","9,26%",3.38,846.00
3,BRADESCO - PRUDENCIAL,1000080075,b1,S1,C,2,OSASCO,SP,06/2024,1.682.417.433,658.552.579,1.521.287.541,1.277.833.567,161.129.891,8.937.210,149.119.407,"15,23%","25,57%",2.61,1.05
4,SANTANDER - PRUDENCIAL,1000080185,b1,S1,C,3,SAO PAULO,SP,06/2024,1.279.020.308,540.556.671,1.180.740.535,900.469.651,98.279.773,6.356.898,98.725.721,"14,38%","20,02%",2.53,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,LAURO DE FREITAS,BA,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,IGACI,AL,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,BEBEDOURO,SP,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,UBERLANDIA,MG,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00


In [65]:
# Verificando o número de colunas e linhas do DataFrame.
print(f'Número de colunas: {df.shape[1]}')
print(f'Número de linhas: {df.shape[0]}')

Número de colunas: 20
Número de linhas: 1357


In [66]:
# Verificando se o Pandas inferiu corretamente os tipos de dados das colunas.
df.dtypes

Instituicao              object
Codigo                    int64
TipoConsBancario         object
Segmento                 object
TipoConsolidacao         object
TipoControle              int64
Cidade                   object
UF                       object
DataBase                 object
AtivoTotal               object
CarteiraCredito          object
PassivoExigivel          object
Captacoes                object
PatrimonioLiquido        object
LucroLiquido             object
PatrimonioReferencia     object
IndiceBasileia           object
IndiceImobilizacao       object
NumAgencias             float64
NumPostosAtendimento    float64
dtype: object

## Transformação e Limpeza dos Dados

Dados reais nem sempre estão prontos para serem analisados. Eles podem conter erros, valores ausentes, formatação inadequada e outras imperfeições que precisam ser tratadas antes de serem utilizadas. Nesta seção, vamos realizar algunas transformações e limpezas nos dados para garantir que eles estejam prontos para a análise exploratória.

In [67]:
# Removendo a parte ' - PRUDENCIAL' da coluna 'Instituicao'.
df['Instituicao'] = df['Instituicao'].str.replace(' - PRUDENCIAL', '')

In [68]:
# A documentação do Banco Central informa que os dados das colunas
# 'TipoConsBancario', 'TipoConsolidacao' e 'TipoControle' são categóricos.
print(f'Tipos de Consolidado Bancário: {df['TipoConsBancario'].unique().tolist()}')
print(f'Segmento Resolução nº 4.553/2017: {df['TipoConsolidacao'].unique().tolist()}')
print(f'Tipo de Consolidação: {df['TipoControle'].unique().tolist()}')

Tipos de Consolidado Bancário: ['b1', 'b4', 'n1', 'n4', 'b2', 'n2', 'b3C', 'b3S']
Segmento Resolução nº 4.553/2017: ['C', 'I']
Tipo de Consolidação: [2, 1, 3]


In [69]:
# A documentação do Banco Central informa que os dados das colunas
# AtivoTotal, CarteiraCredito, PassivoExigivel, Captacoes, PatrimonioLiquido
# e LucroLiquido são numéricos.
print(f'Exemplo de dados da coluna Ativo Total: {df['AtivoTotal'].loc[0]}')
print(f'Exemplo de dados da coluna Carteira de Crédito Classificada: {df['CarteiraCredito'].loc[0]}')
print(f'Exemplo de dados da coluna Passivo Circulante: {df['PassivoExigivel'].loc[0]}')
print(f'Exemplo de dados da coluna Captações: {df['Captacoes'].loc[0]}')
print(f'Exemplo de dados da coluna Patrimônio Líquido: {df['PatrimonioLiquido'].loc[0]}')
print(f'Exemplo de dados da coluna Lucro Líquido: {df['LucroLiquido'].loc[0]}')

Exemplo de dados da coluna Ativo Total: 2.636.882.506
Exemplo de dados da coluna Carteira de Crédito Classificada: 946.459.806
Exemplo de dados da coluna Passivo Circulante: 2.443.588.602
Exemplo de dados da coluna Captações: 1.884.371.314
Exemplo de dados da coluna Patrimônio Líquido: 193.293.904
Exemplo de dados da coluna Lucro Líquido: 19.963.578


In [70]:
# A documentação do Banco Central informa que os dados das colunas
# 'IndiceBasileia' e 'IndiceImobilizacao' são numéricos, representando um percentual.
print(f'Exemplo de dados da coluna Índice de Basileia: {df['IndiceBasileia'].loc[0]}')
print(f'Exemplo de dados da coluna Índice de Imobilização: {df['IndiceImobilizacao'].loc[0]}')

Exemplo de dados da coluna Índice de Basileia: 16,56%
Exemplo de dados da coluna Índice de Imobilização: 20,86%


In [71]:
def clean_number_format(column: pd.Series) -> pd.Series:
    """
    Limpa a formatação de números em uma coluna do DataFrame.
    
    Parameters:
        column (pandas.Series): A coluna a ser limpa.
    """
    
    # Remover o ponto como separador de milhares e substituir a vírgula por ponto decimal.
    return column.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)

In [72]:
def clean_percentage_format(column: pd.Series) -> pd.Series:
    """
    Limpa a formatação de porcentagens em uma coluna do DataFrame.
    
    Parameters:
        column (pandas.Series): A coluna a ser limpa.
    """
    
    # Remover o símbolo de porcentagem e substituir a vírgula por ponto decimal.
    return column.str.replace('%', '', regex=False).str.replace(',', '.', regex=False)

In [73]:
def change_dtype(data_frame: pd.DataFrame,
                 column: str,
                 new_dtype: str) -> pd.DataFrame:
    """
    Altera o tipo de dados de uma coluna do DataFrame.
    
    Parameters:
        data_frame (pandas.DataFrame): O DataFrame contendo os dados.
        column (str): O nome da coluna a ser alterada.
        new_dtype (type): O novo tipo de dados da coluna.
    
    Returns:
        pandas.DataFrame: O DataFrame com a coluna alterada.
    """

    # Mapeamento de tipos especiais
    special_types = {
        'float': lambda col: pd.to_numeric(clean_number_format(col),
                                           errors='coerce'),
        'int': lambda col: col.astype('int'),
        'datetime': lambda col: pd.to_datetime(col,
                                               errors='coerce'),
        'category': lambda col: col.astype('category'),
        'bool': lambda col: col.astype(bool),
        'str': lambda col: col.astype(str)
    }

    try:
        if new_dtype in special_types and new_dtype == 'float':
            if data_frame[column].dtype == 'object' and data_frame[column].str.contains('%').any():
                data_frame[column] = special_types[new_dtype](clean_percentage_format(data_frame[column]))
            else:
                data_frame[column] = special_types[new_dtype](data_frame[column])
        elif new_dtype in special_types:
            data_frame[column] = special_types[new_dtype](data_frame[column])
        else:
            data_frame[column] = data_frame[column].astype(new_dtype)
    except Exception as error:
        print(f"Erro ao converter a coluna '{column}' para {new_dtype}: {error}")

    return data_frame

In [74]:
# Alterando as colunas com dados categóricos para o tipo 'category'.
df = change_dtype(df, 'TipoConsBancario', 'category')
df = change_dtype(df, 'Segmento', 'category')
df = change_dtype(df, 'TipoConsolidacao', 'category')
df = change_dtype(df, 'TipoControle', 'category')

# Alterando a coluna 'DataBase' para o tipo 'datetime'.
df = change_dtype(df, 'DataBase', 'datetime')

# Alterando as colunas com dados numéricos para os tipos 'float'.
df = change_dtype(df, 'AtivoTotal', 'float')
df = change_dtype(df, 'CarteiraCredito', 'float')
df = change_dtype(df, 'PassivoExigivel', 'float')
df = change_dtype(df, 'Captacoes', 'float')
df = change_dtype(df, 'PatrimonioLiquido', 'float')
df = change_dtype(df, 'LucroLiquido', 'float')
df = change_dtype(df, 'PatrimonioReferencia', 'float')

# Alterando as colunas com dados percentuais para o tipo 'float'.
df = change_dtype(df, 'IndiceBasileia', 'float')
df = change_dtype(df, 'IndiceImobilizacao', 'float')

# Alterando as colunas com dados numéricos para os tipos 'int'.
df = change_dtype(df, 'NumAgencias', 'int')
df = change_dtype(df, 'NumPostosAtendimento', 'int')

In [75]:
# Para facilitar compreensão da dimensionalidade dos dados,
# vamos converter os valores monetários para milhões.

# Função lambda para converter os valores para milhões.
change_to_millions = lambda column: column / 1_000

# Aplicando a função lambda para as colunas de valores monetários.
df['AtivoTotal'] = df['AtivoTotal'].apply(change_to_millions)
df['CarteiraCredito'] = df['CarteiraCredito'].apply(change_to_millions)
df['PassivoExigivel'] = df['PassivoExigivel'].apply(change_to_millions)
df['PatrimonioLiquido'] = df['PatrimonioLiquido'].apply(change_to_millions)
df['LucroLiquido'] = df['LucroLiquido'].apply(change_to_millions)

In [76]:
# Alterando as informações da coluna Cidade para que todas as palavras comecem com letra maiúscula.
df['Cidade'] = df['Cidade'].str.title()

In [77]:
# Criando uma nova coluna 'Regiao' com base na coluna 'UF'.
# Obtendo a posição da coluna 'Estado'.
posicao_estado = df.columns.get_loc('UF')
df.insert(posicao_estado, 'Regiao', df['UF'].map(estado_para_regiao))
# Alterando o tipo de dados da coluna 'Regiao' para 'category'.
df = change_dtype(df, 'Regiao', 'category')

In [78]:
# Identificando as colunas que possuem valores 'NI (Não Informado)'.
df.eq('NI').sum().sort_values(ascending=False).rename('NI Count')

Instituicao             0
Codigo                  0
TipoConsBancario        0
Segmento                0
TipoConsolidacao        0
TipoControle            0
Cidade                  0
Regiao                  0
UF                      0
DataBase                0
AtivoTotal              0
CarteiraCredito         0
PassivoExigivel         0
Captacoes               0
PatrimonioLiquido       0
LucroLiquido            0
PatrimonioReferencia    0
IndiceBasileia          0
IndiceImobilizacao      0
NumAgencias             0
NumPostosAtendimento    0
Name: NI Count, dtype: int64

In [79]:
# Verificando a existência de valores nulos no DataFrame.
df.isnull().sum().sort_values(ascending=False).rename('Null Count')

IndiceImobilizacao      134
PatrimonioReferencia     39
IndiceBasileia           39
CarteiraCredito          29
Captacoes                29
PatrimonioLiquido        29
PassivoExigivel          29
AtivoTotal               29
LucroLiquido             29
Segmento                  2
TipoConsBancario          0
Codigo                    0
Instituicao               0
TipoConsolidacao          0
DataBase                  0
Regiao                    0
UF                        0
TipoControle              0
Cidade                    0
NumAgencias               0
NumPostosAtendimento      0
Name: Null Count, dtype: int64

In [80]:
# Filtrando apenas as linhas que possuem 'NaN' como valor.
df[df.isna().any(axis=1)]

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,Regiao,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
9,NU PAGAMENTOS,1000084693,n1,S3,C,2,Sao Paulo,Sudeste,SP,2024-06-01,214500.65,101837.59,200136.12,134684439.00,14364.53,4069.33,16574240.00,1581.00,NaN,8,0
16,CIELO IP,1000084710,n4,S5,C,2,Barueri,Sudeste,SP,2024-06-01,100538.99,888.42,80147.54,3667690.00,20391.46,1038.62,9116259.00,3454.00,NaN,1,0
22,PAGSEGURO,1000084813,b1,S4,C,2,Sao Paulo,Sudeste,SP,2024-06-01,66937.97,2460.70,55598.37,40615403.00,11339.60,774.18,9491445.00,3336.00,NaN,6,0
24,BANCO MASTER,1000080367,b1,S3,C,2,Rio De Janeiro,Sudeste,RJ,2024-06-01,57152.26,27542.99,52550.65,49522848.00,4601.60,433.03,NaN,NaN,NaN,16,5
27,MERCADO PAGO IP,1000084820,n1,S4,C,2,Osasco,Sudeste,SP,2024-06-01,46794.94,10188.45,43786.93,24492888.00,3008.01,349.11,2626620.00,1451.00,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,Lauro De Freitas,Nordeste,BA,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,Igaci,Nordeste,AL,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,Bebedouro,Sudeste,SP,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,Uberlandia,Sudeste,MG,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [81]:
# Substituindo os valores 'NaN' por NA (Not Available).
df = df.fillna(pd.NA)

In [82]:
# Verificando como ficou o DataFrame após as transformações.
df

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,Regiao,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
0,ITAU,1000080099,b1,S1,C,2,Sao Paulo,Sudeste,SP,2024-06-01,2636882.51,946459.81,2443588.60,1884371314.00,193293.90,19963.58,215556905.00,1656.00,2086.00,2,1
1,BB,1000080329,b1,S1,C,1,Brasilia,Centro-oeste,DF,2024-06-01,2334367.94,1024504.85,2161635.72,1845389366.00,172732.22,18173.03,175348301.00,1419.00,1635.00,4,512
2,CAIXA ECONÔMICA FEDERAL,1000080738,b1,S1,C,1,Brasilia,Centro-oeste,DF,2024-06-01,1907044.01,1175742.24,1807735.64,1617770675.00,99308.37,5300.68,129887763.00,1640.00,926.00,3,846
3,BRADESCO,1000080075,b1,S1,C,2,Osasco,Sudeste,SP,2024-06-01,1682417.43,658552.58,1521287.54,1277833567.00,161129.89,8937.21,149119407.00,1523.00,2557.00,2,1
4,SANTANDER,1000080185,b1,S1,C,3,Sao Paulo,Sudeste,SP,2024-06-01,1279020.31,540556.67,1180740.53,900469651.00,98279.77,6356.90,98725721.00,1438.00,2002.00,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,Lauro De Freitas,Nordeste,BA,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,Igaci,Nordeste,AL,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,Bebedouro,Sudeste,SP,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,Uberlandia,Sudeste,MG,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
